# Exporting Training Data, Training a model and Detecting Object using Arcpy

### Preparing the Environment

In [1]:
# Importing the required modules
import arcpy
import os
from arcpy.ia import TrainDeepLearningModel, DetectObjectsUsingDeepLearning

In [2]:
# Setting up the environment
arcpy.env.workspace = "path to the workspace"
arcpy.env.overwriteOutput = True # Else, user will get folder already exists and using arcpy in ArcGIS Pro

In [3]:
# Use the following if using the notebook within ArcGIS Pro environment
# aprx = arcpy.mp.ArcGISProject('CURRENT')

# If running outside ArcGIS Pro environemnt
aprx = arcpy.mp.ArcGISProject("path to the arcgis pro .aprx")

### Referencing the map and getting the layers in the map

In [4]:
reference_map = aprx.listMaps()[0]# As there is only one map in the project file.

# Getting list of all layers in the map
map_layers = [layers for layers in reference_map.listLayers()]

map_layers_name = [layers.name for layers in reference_map.listLayers()]
map_layers_name

['TrainingSamplesComplete',
 'Detected_Swimming_Pools',
 'NAIP_AOI.tif',
 'Topographic']

In [5]:
# Creating the layer object for use in Export Training Data tool and specifying the output location
naip_aoi = [l for l in map_layers if l.name=='NAIP_AOI.tif'][0]
training_sample = [l for l in map_layers if l.name=='TrainingSamplesComplete'][0]
class_value_field_name = [field.name for field in arcpy.ListFields(training_sample)][2]

In [6]:
# Checking Out Image Analyst extension that is required.
from arcpy.ia import ExportTrainingDataForDeepLearning

# Check out the ArcGIS Image Analyst extension license
arcpy.CheckOutExtension("ImageAnalyst")

'CheckedOut'

### Exporting Training Data

In [7]:
out_folder = os.path.join(os.getcwd(), "Training_Images_arcpy")

# Setting local variables as per the exercise
image_chip_format = "TIFF"
tile_size_x = "256"
tile_size_y = "256"
stride_x= "128"
stride_y= "128"
output_nofeature_tiles = "ONLY_TILES_WITH_FEATURES"
metadata_format= "PASCAL_VOC_rectangles"
buffer_radius = 6
rotation_angle = 0
reference_system = "MAP_SPACE"

# Exporting Training Data using arcpy
ExportTrainingDataForDeepLearning(naip_aoi, out_folder, in_class_data=training_sample, \
                                  image_chip_format=image_chip_format, tile_size_x=tile_size_x, tile_size_y=tile_size_y, \
                                 stride_x = stride_x, stride_y=stride_y, metadata_format=metadata_format, rotation_angle=rotation_angle, \
                                 buffer_radius=buffer_radius, class_value_field=class_value_field_name, reference_system=reference_system)


<geoprocessing server result object at 0x210bc7a7fa8>

### Exporting Training Data Completed

# Time to train the model

In [11]:
def get_extent(lyr):
    """
    Function to calculate the extent of a layer.
    return: extent coordinates as string
    """
    if lyr.supports('DATASOURCE'):
        desc = arcpy.Describe(lyr.dataSource)
        extent = f"{desc.extent}"
        split_extent = extent.split(' ')[0:4]
        concat_extent = ' '.join(split_extent)
        return concat_extent
    else:
        return None

In [12]:
training_sample_extent = get_extent(training_sample)

# Setting up local variables for training the models
input_folder = out_folder
output_folder = os.path.join(os.getcwd(), "Trained_SSD_Model")
max_epochs = 25
model_type = "SSD"
batch_size = 32
learning_rate = None
backbone_model = "resnet34"
pretrained_model = None
validation_percentage = 20 # I chose 20 instead of 10 in the exercise
stop_training = "STOP_TRAINING"
freeze = "FREEZE_MODEL"

# Training the model
with arcpy.EnvManager(parallelProcessingFactor="12", extent=training_sample_extent, processorType="GPU"):
    TrainDeepLearningModel(in_folder=input_folder, out_folder=output_folder, max_epochs=max_epochs, batch_size=batch_size,\
                          model_type=model_type, learning_rate=learning_rate, backbone_model=backbone_model, \
                          pretrained_model=pretrained_model, stop_training=stop_training, freeze=freeze)

epoch,train_loss,valid_loss,time
0,5042.982422,4583.544434,00:36
1,3534.752197,1062.215210,00:36
2,1893.369141,533.428162,00:36
3,1074.566528,451.426178,00:36
4,698.143005,417.292877,00:35
5,516.629639,397.246674,00:35
6,423.896271,395.837402,00:36
7,370.732208,374.510986,00:36
8,344.694641,353.733215,00:36
9,322.871948,360.573212,00:36


The output displays the training loss and the validation loss during each epoch

### Training Completed and Model exported

# Detecting Swimming Pool

In [17]:
# Setting up local variable for detecing objects, same as in the exercise
detected_objects_location = os.path.join(os.getcwd(), "ObjectDetection.gdb/Detected_Pools_arcpy") 
model = os.path.join(output_folder, "Trained_SSD_Model.emd")
arguments = "padding 56;threshold 0.5;nms_overlap 0.1;batch_size 64;exclude_pad_detections True"
run_nms = "NMS"
confidence_score_field = "Confidence"
class_value_field = "Class"
max_overlap_ratio = 0


with arcpy.EnvManager(parallelProcessingFactor="12", extent=training_sample_extent, processorType="GPU"):
    DetectObjectsUsingDeepLearning(in_raster=naip_aoi, out_detected_objects=detected_objects_location, in_model_definition=model, \
                                   arguments = arguments, run_nms=run_nms,confidence_score_field=confidence_score_field, \
                                   class_value_field=class_value_field, max_overlap_ratio=max_overlap_ratio)


AttributeError: 'ArcGISObjectDetector' object has no attribute 'updatePixels'

#### Despite the above error, features depicting the detected swimming pool was written to the file geodatabase.
#### Any hints as to why the above error appeared would be helpful.